## Data loading and saving

In [ ]:
from psd_tools import PSDImage
from PIL import Image
from dotenv import load_dotenv

import sys
sys.path.append('../src')

import numpy as np
import io
import glob
import utils.constant as constant
import os
import matplotlib.pyplot as plt

%matplotlib inline

### Set input and output directories

In [ ]:
load_dotenv()

input_dir = os.getenv('INPUT_DIR')
im_dir = os.getenv('IMAGE_DIR')
gt_dir = os.getenv('MASK_DIR')

filenames = glob.glob(input_dir, recursive=True)
num_images = len(filenames)

### Read photoshop files and save as numPy arrays

In [ ]:
# Create output directories 
if not os.path.exists(im_dir):
    os.makedirs(im_dir)
if not os.path.exists(gt_dir):
    os.makedirs(gt_dir)

num_images = len(filenames)
i = 0
for filename in filenames:
    im = []
    mask = []
    gt = []
    
    # Print status
    if i % 5 == 0:
        print(i, '/', num_images, 'images processed')

    # Open photoshop image
    try:
        psd = PSDImage.open(filename)
    except:
        continue
        
    im_name = filename.split('/')[-1].split('.')[0]+'.npy'
    
    # Iterate through individual layers
    for layer in psd:
        
        # Convert image to PIL Image
        if layer.name == constant.ORIGINAL:
            im = np.array(layer.topil())  
            dim = im.shape
            
        # Convert mask to PIL Image
        elif layer.name == constant.MASK:
            try:
                mask = np.array(layer.topil())[:,:,3]/255.  # Select only last channel and normalize
            except:
                continue

            # Construct mask that matches image dimensions - input mask is clipped and needs padding
            mask_height = mask.shape[0]
            gt = np.zeros((dim[0],dim[1]))
            offset = layer.offset[1]

            # Adjust mask location by # of offset pixels in y direction
            try:
                gt[offset:offset+mask_height,:] = mask
            except:
                print(i,'non-matching dimensions')
                gt = [] 

    # Serialize images as numPy if there exists a valid image and mask
    if len(im) > 0 and len(gt) > 0:
        np.save(os.path.join(im_dir,im_name), im)
        np.save(os.path.join(gt_dir,im_name), gt)
        
    i+=1